In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Extract

The first file we are going to be using is a list of breweries in the US. This csv has columns for name, address, website, establishment type, state, and number of breweries in that state.
The second file we are going to be using is also a csv of state population data from the US census. This file also contains columns giving the latitude and longitude of the given state's capital. Both of these datasets were available on Kaggle.com.

In [2]:
pop_file = "Resources/2019_Census_US_Population_Data_By_State_Lat_Long.csv"
pop_df = pd.read_csv(pop_file)
pop_df.head()

,STATE,POPESTIMATE2019,lat,long
0,Alabama,4903185,32.377716,-86.300568
1,Alaska,731545,58.301598,-134.420212
2,Arizona,7278717,33.448143,-112.096962
3,Arkansas,3017804,34.746613,-92.288986
4,California,39512223,38.576668,-121.493629
